In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!pip install "ray[tune]"

import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer)
import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix
os.environ["WANDB_DISABLED"] = "true"

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.6 MB/s 
     |████████████████████████████████| 7.6 MB 52.7 MB/s 
     |████████████████████████████████| 163 kB 60.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.3 MB/s 
     |████████████████████████████████| 8.8 MB 61.0 MB/s 
     |████████████████████████████████| 125 kB 70.7 MB/s 
     |████████████████████████████████| 468 kB 50.0 MB/s 


In [ ]:
no_train_epochs = 4
freeze_layer_count = 4
pretrained_model_tokenizer_path = r""
df_input = pd.read_csv(r"")
df_input_val = pd.read_csv(r"")
df_test = pd.read_csv(r"")
model_path = ""


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_tokenizer_path)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred[0], axis=1)

    print(confusion_matrix(labels, pred))

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_tokenizer_path, num_labels=2)

'''for param in model.roberta.parameters():
    param.requires_grad = False'''

for layer in model.roberta.encoder.layer[:freeze_layer_count]:
    for param in layer.parameters():
        param.requires_grad = False

df_input = df_input.sample(frac=1)

text_train = list(df_input['text'])
label_train = list(df_input['label'])

df_input_val = df_input_val.sample(frac=1)

text_val = list(df_input_val['text'])
label_val = list(df_input_val['label'])

#text_train, text_val, label_train, label_val = train_test_split(text, label, test_size=0.2)
text_train_tokenized = tokenizer(text_train, padding=True, truncation=True, max_length=100)
text_val_tokenized = tokenizer(text_val, padding=True, truncation=True, max_length=100)

train_dataset = Dataset(text_train_tokenized, label_train)
val_dataset = Dataset(text_val_tokenized, label_val)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        pretrained_model_tokenizer_path, return_dict=True)
    
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    eval_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=no_train_epochs,
    seed=200,
)

#args = TrainingArguments("test", save_strategy="epoch", save_total_limit=1, evaluation_strategy="steps", eval_steps=500, disable_tqdm=True)

trainer = Trainer(
    #model_init=model_init,
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

'''best_trial = trainer.hyperparameter_search(
            backend="ray",
            direction='maximize',
            n_trials=10,
       )'''

#RUNNING    | 172.28.0.2:9174 |     1.12076e-05 |                  4 |                     16 |  1.89943 
trainer.train()
trainer.save_model(model_path)

# Create torch dataset
df_test = df_test.sample(frac=1)

text_test = list(df_test['text'])
label_test = list(df_test['label'])

text_test_tokenized = tokenizer(text_test, padding=True, truncation=True, max_length=100)
test_dataset = Dataset(text_test_tokenized)

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred[0], axis=1)

print("Testing done")
print(y_pred)
print("Confusion Matrix:")
print(confusion_matrix(label_test, y_pred))

test_f1 = f1_score(y_true=label_test, y_pred=y_pred, average='macro')
test_accuracy = accuracy_score(y_true=label_test, y_pred=y_pred)
test_recall = recall_score(y_true=label_test, y_pred=y_pred, average='macro')
test_precision = precision_score(y_true=label_test, y_pred=y_pred, average='macro')

print("Test scores")
print("Accuracy: {}\nF1: {}\nPrecision: {}\nRecall: {}\n".format(test_accuracy, test_f1, test_precision, test_recall))

Some weights of the model checkpoint at /content/drive/MyDrive/Thesis/model_dr_ksdt_ht_e20 were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Thesis/model_dr_ksdt_ht_e20 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.306800,0.821879,0.888000,0.889769,0.880372,0.884075
2,0.033200,0.965199,0.864000,0.860329,0.862028,0.861120
3,0.019200,0.931871,0.864000,0.860329,0.862028,0.861120
4,0.009500,0.989421,0.872000,0.868182,0.871462,0.869588


***** Running Evaluation *****
  Num examples = 125
  Batch size = 8


[[67  5]
 [ 9 44]]


***** Running Evaluation *****
  Num examples = 125
  Batch size = 8


...

[[63  9]
 [ 8 45]]


***** Running Evaluation *****
  Num examples = 125
  Batch size = 8


[[63  9]
 [ 7 46]]




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs
Configuration saved in /content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs/config.json
Model weights saved in /content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs/pytorch_model.bin
loading configuration file /content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_po

Testing done
[1 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0
 1 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 0 0 1 0 1 0 0 0 1]
Confusion Matrix:
[[73  7]
 [ 9 36]]
Test scores
Accuracy: 0.872
F1: 0.8597081930415265
Precision: 0.8637266023823029
Recall: 0.85625



In [ ]:
pretrained_model_tokenizer_path = r""

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_tokenizer_path)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

model_path = ""

df_test_adv_neg = pd.read_csv(r"")

text_test_adv_neg = list(df_test_adv_neg['text'])
label_test_adv_neg = list(df_test_adv_neg['label'])

text_test_adv_neg_tokenized = tokenizer(text_test_adv_neg, padding=True, truncation=True, max_length=100)
test_dataset_adv_neg = Dataset(text_test_adv_neg_tokenized)

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Define test trainer
test_adv_neg_trainer = Trainer(model)

# Make prediction
raw_pred_adv_neg, _, _ = test_adv_neg_trainer.predict(test_dataset_adv_neg)

# Preprocess raw predictions
y_pred_adv_neg = np.argmax(raw_pred_adv_neg[0], axis=1)

'''print(raw_pred_adv_neg)
print("Testing done")
print(y_pred_adv_neg)'''
print("Confusion Matrix:")
print(confusion_matrix(label_test_adv_neg, y_pred_adv_neg))

test_adv_neg_f1 = f1_score(y_true=label_test_adv_neg, y_pred=y_pred_adv_neg, average='macro')
test_adv_neg_accuracy = accuracy_score(y_true=label_test_adv_neg, y_pred=y_pred_adv_neg)
test_adv_neg_recall = recall_score(y_true=label_test_adv_neg, y_pred=y_pred_adv_neg, average='macro')
test_adv_neg_precision = precision_score(y_true=label_test_adv_neg, y_pred=y_pred_adv_neg, average='macro')

print("Test scores For Negation")
print("Accuracy: {}\nF1: {}\nPrecision: {}\nRecall: {}\n".format(test_adv_neg_accuracy, test_adv_neg_f1, test_adv_neg_precision, test_adv_neg_recall))

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /content/drive/MyDrive/Thesis/finetuned_dr_ksdt_het_contrastive_20_4_epochs/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/Thesis/finetuned_dr_ksdt_het_contrastive_20_4_epochs",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_

Confusion Matrix:
[[70 10]
 [ 9 36]]
Test scores For Negation
Accuracy: 0.848
F1: 0.8358559679314397
Precision: 0.8343423225096313
Recall: 0.8375



In [ ]:
model_path = ""

df_test_adv_spell = pd.read_csv(r"")

text_test_adv_spell = list(df_test_adv_spell['text'])
label_test_adv_spell = list(df_test_adv_spell['label'])

text_test_adv_spell_tokenized = tokenizer(text_test_adv_spell, padding=True, truncation=True, max_length=100)
test_dataset_adv_spell = Dataset(text_test_adv_spell_tokenized)

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Define test trainer
test_adv_spell_trainer = Trainer(model)

# Make prediction
raw_pred_adv_spell, _, _ = test_adv_spell_trainer.predict(test_dataset_adv_spell)

# Preprocess raw predictions
y_pred_adv_spell = np.argmax(raw_pred_adv_spell[0], axis=1)

print("Testing done")
print(y_pred_adv_spell)
print("Confusion Matrix:")
print(confusion_matrix(label_test_adv_spell, y_pred_adv_spell))

test_adv_spell_f1 = f1_score(y_true=label_test_adv_spell, y_pred=y_pred_adv_spell, average='macro')
test_adv_spell_accuracy = accuracy_score(y_true=label_test_adv_spell, y_pred=y_pred_adv_spell)
test_adv_spell_recall = recall_score(y_true=label_test_adv_spell, y_pred=y_pred_adv_spell, average='macro')
test_adv_spell_precision = precision_score(y_true=label_test_adv_spell, y_pred=y_pred_adv_spell, average='macro')

print("Test scores for Spelling")
print("Accuracy: {}\nF1: {}\nPrecision: {}\nRecall: {}\n".format(test_adv_spell_accuracy, test_adv_spell_f1, test_adv_spell_precision, test_adv_spell_recall))

loading configuration file /content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/Thesis/finetuned_dr_ksdt_ht_contrastive_20_4_epochs",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/

Testing done
[0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 1 1
 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1 1
 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1
 0 0 1 1 1 1 0 0 0 1 0 0 1 1]
Confusion Matrix:
[[70 10]
 [ 8 37]]
Test scores for Spelling
Accuracy: 0.856
F1: 0.8452118877270226
Precision: 0.8423349699945445
Recall: 0.8486111111111111

